<h2> Spectroscopic data reduction : spectra module

In [129]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [130]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [131]:
indir='/home/users/adijeau/Documents/UT220107'
red=imred.Reducer('KOSMOS',dir=indir)

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/users/adijeau/Documents/UT220107/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [ 5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a CCDData object, and numpy array, or a FITS HDU object.

In [132]:
a=red.reduce(22)

t.tv(a)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_GCPilot02_red.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field.
<br>
First, however, we have to make the flat field, which is accomplished using the mkflat() method, which takes as input a list of frames to be used to construct the master flat field (superflat). For a spectrograph, we use the spec=True keyword which will remove the spectral signature from the combined flat.

In [133]:
flatims=[61,62,63,64,65]
flat=red.mkflat(flatims,spec=True,display=None)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0061.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0062.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0063.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0064.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Flat_red.0065.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with median....
  calculating uncertainty....


Read and display a star spectral image

In [134]:
star=red.reduce(2,flat=flat)
t.tv(star,max=1000) 

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0002.fits
  subtracting overscan vector 
  subtracting overscan vector 
  flat fielding...
INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [144]:
crstar=red.crrej(star,crbox=[9,1],display=t)

  zapping CRs with filter [9,1]...
  See CRs and CR-zapped image and original using - key
   To continue, hit space in display window (p for debug) 


In [145]:
t.tv(crstar.bitmask,min=0,max=2)


<h4> Tracing and extraction

We want to extract the spectrum, i.e. from the 2D image to a 1D spectrum. Start by defining a trace that is just constant along rows, and extract the spectrum with a 25 pixel window.

In [8]:
# here we set a trace to be at a constant position on the detector
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
vars(trace) 

{'type': 'Polynomial1D',
 'degree': 2,
 'sc0': 2048,
 'pix0': 0,
 'spectrum': array([    0.,     0.,     0., ...,  3448.,  3443.,  3442.]),
 'rad': 5,
 'lags': array([-300, -299, -298, -297, -296, -295, -294, -293, -292, -291, -290,
        -289, -288, -287, -286, -285, -284, -283, -282, -281, -280, -279,
        -278, -277, -276, -275, -274, -273, -272, -271, -270, -269, -268,
        -267, -266, -265, -264, -263, -262, -261, -260, -259, -258, -257,
        -256, -255, -254, -253, -252, -251, -250, -249, -248, -247, -246,
        -245, -244, -243, -242, -241, -240, -239, -238, -237, -236, -235,
        -234, -233, -232, -231, -230, -229, -228, -227, -226, -225, -224,
        -223, -222, -221, -220, -219, -218, -217, -216, -215, -214, -213,
        -212, -211, -210, -209, -208, -207, -206, -205, -204, -203, -202,
        -201, -200, -199, -198, -197, -196, -195, -194, -193, -192, -191,
        -190, -189, -188, -187, -186, -185, -184, -183, -182, -181, -180,
        -179, -178, -177, -

In [9]:
trace.model[0](2048)
plt.figure()
plt.plot(trace.model[0](np.arange(0,4096)))

We can use the existing trace to retrace a new spectrum. This is done by cross-correlating the reference spectrum with a spatial cut across the input spectrum (at sc0) to find the shift, then using the old model to start the trace from that starting position. The functional form of the old model is preserved:

In [10]:
trace.retrace(star,display=t)

Using shift:  12.630110042


/home/holtz/git/pyvista/python/pyvista/spectra.py:907: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:912: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<1))[0]



  See trace. Hit space bar to continue....


Alternatively, one could use the find() method to determine the shift between the stored trace and your frame, then just go straight to extraction using the shape of the stored trace; this would be applicable if your object is too faint, or doesn't have sufficient continuum, to trace.


You can alos just find peak(s) in the spatial direction using findpeak(), then trace them:

In [11]:
peaks,fibers=trace.findpeak(star,thresh=200)
print(peaks)
print(fibers)
                            
trace.trace(star,peaks,display=t)

looking for peaks using 200 pixels around 2048, threshhold of 200.000000
[957]
[0]


/home/holtz/git/pyvista/python/pyvista/spectra.py:907: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:912: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<1))[0]



  See trace. Hit space bar to continue....


The model is now modified for the input spectrum:

In [12]:
trace.model[0](2048)

956.9233851837214

If you needed to create Trace from scratch, see below. Note the skip= keyword which can be used when tracing to speed things up by only computing centroids every skip pixels, taking a median around these pixels of width skip pixels. The default is skip=10

In [19]:
trace=spectra.Trace(sc0=2048,lags=range(-300,300),
                    rows=[550,1450],transpose=red.transpose)
vars(trace)
srow=[955]   # list to allow for multiple spectra on an image
starec=trace.trace(star,srow,display=t,skip=20) 
vars(trace)

/home/holtz/git/pyvista/python/pyvista/spectra.py:908: RuntimeWarning: invalid value encountered in true_divide
  model=(fitter(mod,cols[gd],ypos[gd]))
/home/holtz/git/pyvista/python/pyvista/spectra.py:913: RuntimeWarning: invalid value encountered in true_divide
  model=(fitter(mod,cols[gd],ypos[gd]))



  See trace. Hit space bar to continue....


{'type': 'Polynomial1D',
 'degree': 2,
 'pix0': 0,
 'spectrum': array([      0.  ,       0.  ,       0.  , ...,  141872.2 ,  142337.6 ,
         142198.83], dtype=float32),
 'rad': 5,
 'transpose': True,
 'rows': [550, 1450],
 'lags': range(-300, 300),
 'sc0': 2048,
 'model': [<Polynomial1D(2, c0=948.7571394431843, c1=0.004432319671951549, c2=-2.248449711639477e-07)>],
 'sigmodel': [<Polynomial1D(2, c0=3.9076653803478174, c1=-0.0006160422635450908, c2=6.064520792204683e-08)>],
 'index': array([0])}

Extraction is done using the extract() method of the Trace object, currently just with simple boxcar extraction. Use rad= to specify window radius. Use back=[[b1,b2],[b3,b4]] to subtract background as determined from one or more background windows (note argument should be list of 2-element lists, giving start and end pixel of each background window).

In [21]:
starec=trace.extract(star,display=t,rad=25)
print(starec.shape)

  See extraction window(s). Hit space bar to continue....

(1, 4096)


extract() returns a CCDData object, with extracted spectrum, and uncertainty. Note that this is a 2D array to accomodate multiple objects/traces, even if there is only a single object.

In [22]:
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec.data[0]/starec.uncertainty.array[0])

In [23]:
star.data+=5000
starec2=trace.extract(star,rad=25,back=[[50,75],[-75,-50]],display=t)
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec2.data[0])
plt.show()

  See extraction window(s). Hit space bar to continue....



<h4> Wavelength calibration

Now let's turn to wavelength calibration, i.e. getting a function that gives the wavelength as a function of pixel. We'll solve for this using arc frames, here taken with each lamp separately, so sum the three exposures

In [24]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([46,47,48])
t.clear()
t.tv(arcs)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_red.0046.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_red.0047.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_red.0048.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with sum....
  calculating uncertainty....


In [25]:
arcec=trace.extract(arcs,display=None,rad=20)

Wavelength calibration first stars with identifying lines. This is much easier if one can work from a previous solution. Here we start by reading a previous solution into a pyvista WaveCal object.

In [26]:
wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
vars(wav)

  rms:    0.177 Angstroms (50 lines)


{'type': 'chebyshev',
 'degree': 3,
 'ydegree': 2,
 'waves': array([ 5562.22534,  5570.28944,  5572.548  ,  5764.418  ,  5852.4878 ,
         5866.75017,  5870.91599,  5875.618  ,  5879.90035,  5944.8342 ,
         5945.45239,  5977.64573,  6029.9971 ,  6074.3377 ,  6096.163  ,
         6143.0623 ,  6163.5939 ,  6217.2813 ,  6266.495  ,  6304.7892 ,
         6334.4279 ,  6382.9914 ,  6402.246  ,  6506.5279 ,  6532.8824 ,
         6598.9529 ,  6678.2    ,  6717.0428 ,  6965.43   ,  7032.4127 ,
         7065.188  ,  7147.041  ,  7173.939  ,  7245.167  ,  7383.98   ,
         7438.899  ,  7635.105  ,  7723.8    ,  7948.175  ,  8082.458  ,
         8264.521  ,  8408.208  ,  8424.647  ,  8521.441  ,  8634.647  ,
         8654.3831 ,  8667.9442 ,  8697.94   ,  8853.866  ,  9122.97   ]),
 'waves_order': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]),
 'orders': array(

With a previous solution loaded, the identify routine will cross correlate the input spectrum with the previous solution, then attempt to identify the lines from the previous solution at the shifted pixel position from the previous solution. Finally, it does a fit.

The identify() method fits Gaussians to the lines to get their position, and gets FWHM in the process. rad= keyword specifies number of pixels on either side of line to use in fit. If you add the file= keyword, then the routine will try to identify all of the lines from the reference file; this might be useful if you wanted to extend the wavelength range of the lines from the initial object, e.g. if you were working at a different grating tilt or slit location. If you did that, you might want to save your WaveCal object after cleaning the line list for good lines, so you could use that as a starting guess for other data taken at a similar wavelength setting.

In [29]:
wav.identify(arcec,plot=True,rad=10) #,file='henearkr.dat')

  See identified lines.
  rms:    0.186 Angstroms (39 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.186 Anstroms
  input from plot window...



Note that once you remove lines, they are removed for subsequent uses of the WaveCal object.

Show the FWHM of the lines as a function of wavelength

In [31]:
wav.fwhm
#t.plotax2.cla()
#t.plotax2.plot(wav.waves,np.abs(wav.fwhm),'ro')

array([  7.51733993,   7.38004035,   8.2377014 ,   7.12029596,
         7.04184462,   7.85202414,   7.05256314,   6.92575138,
         7.69522282,   6.75912605,   7.32420472,   7.55680496,
         7.14689568,   7.02902312,   7.16932955,   7.95083635,
        10.35116407,   6.60069658,   7.77485251,   7.27332913,
         7.55196046,   7.35871043,   6.98944463,   7.64774241,
         6.95735871,   7.12522428,   5.64886978,   7.77167227,
         7.04260417,   7.04207638,   6.3372893 ,   8.05942541,
         6.49702256,   7.41045729,   7.07522849,   7.49722027,
         7.22778797,   7.53205141,   7.62227284])

OK, now use the wavelength solution to get wavelength as a function of pixel. We can save that in the wave attribute of our Data object.

In [62]:
wav.add_wave(starec)
plt.figure()
plt.plot(starec.wave[0],starec.data[0])

# get inverse relation, i.e. pixels as f(wavelength)
pix=np.arange(4096)
from scipy.interpolate import CubicSpline
wav2pix=CubicSpline(np.flip(starec.wave[0]),np.flip(pix))

Next cell has resampling of spectrum onto a uniform wavelength grid

In [64]:
plt.figure()
plt.clf()
plt.plot(starec.wave[0],starec.data[0])
print(1/(5.5e-5*np.log(10)))
wnew=10**np.arange(3.5,4.0,5.5e-6)
plt.plot(wnew,wav.scomb(starec,wnew).data)

7896.26330733


/home/holtz/git/pyvista/python/pyvista/spectra.py:649: RuntimeWarning: invalid value encountered in true_divide
  Uses current wavelength solution, linearly interpolates to specified
/home/holtz/git/pyvista/python/pyvista/spectra.py:650: RuntimeWarning: divide by zero encountered in true_divide
  wavelengths, on a row-by-row basis.


<h4> Adjusting wavelength solution for flexure

Next cell shows some possibilities for adjusting wavelength solution based on sky lines

In [66]:
objec=trace.extract(red.reduce(22),display=None,rad=20)
wav.add_wave(objec)


  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_GCPilot02_red.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 



In [67]:
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

If we fix all but the first term, we solve for a constant wavelength shift

In [68]:
import copy
swav=copy.copy(wav)
wave=wav.wave(image=objec.shape)
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(wav.model)
print(swav.model)

/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    3.885 Angstroms (12 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    3.885 Anstroms
  input from plot window...
  rms:    3.164 Anstroms
  input from plot window...
  rms:    0.295 Anstroms
  input from plot window...

Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 3
Domain: (478.7846294613704, 4025.021091218613)
Parameters:
          c0            c1             c2            c3     
    ------------- -------------- ------------- -------------
    7297.34248993 -1793.36403891 45.9677386179 12.5784329687
Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 3
Domain: (478.7846294613704, 4025.021091218613)
Parameters:
          c0            c1             c2            c3     
    ------------- -------------- -----------

<h4> Flux calibration

Instantiate a FluxCal object. You can set it up to use a polynomial fit to the response curve by specifying the polynomial degree with the degree= keyword. If you set degree=-1, the response curve will be determined by a median of the individual response curves, optionally smoothed with a median filter over wavelength (see response() method below).

In [74]:
flx=spectra.FluxCal(degree=-1)
polyflx=spectra.FluxCal(degree=5)

Load in several extracted spectra of flux standards, here all of Feige 34. The standard star spectrum is given using the file= keyword, where the input file should have labelled columns wave, flux, and bin. Alternatively, you can pass an astropy Table with (at least) these three columns, using the stdflux= keyword

In [75]:
for im in [2,3, 38,39] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    wav.add_wave(starec)
    flx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')
    polyflx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0002.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  -1.19505020682


/home/holtz/git/pyvista/python/pyvista/spectra.py:943: RuntimeWarning: invalid value encountered in true_divide
  model=(fitter(mod,cols[gd],ygpos[gd]))
/home/holtz/git/pyvista/python/pyvista/spectra.py:952: RuntimeWarning: invalid value encountered in true_divide
  




  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0003.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  0.668945796134
  Tracing row: 957

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0038.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  2.05481929936
  Tracing row: 959

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0039.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  -1.26032688424
  Tracing row: 958



Solve for the median response curve, taking a median filter over this median.

In [77]:
flx.response(plot=True,medfilt=200)
polyflx.response(plot=True)

Apply the response curve to an object. Note that, using Data objects, the S/N is preserved!

In [82]:
fig,ax=plots.multi(1,2,hspace=0.001)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])
flx.correct(starec,starec.wave)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])

<h3> longslit extraction and wavelength calibration

For extended objects, and perhaps for sky subtraction, we might want to work along the slit. The wavelength solution varies along the slit (line curvature), usually with more than just an offset.

Start by working along the slit to identify lines

In [84]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arc2d=trace.extract2d(arcs)
t=tv.TV()
t.tv(arc2d)

In [86]:
from pyvista import image
wav=spectra.WaveCal(file='KOSMOS/KOSMOS_red_waves.fits')
image.smooth(arc2d,[5,1])
t.clear()
t.tv(arc2d)
wav.identify(arc2d, rad=10,display=None, plot=True,
              nskip=20,lags=np.arange(-10,10))
wav.add_wave(arc2d)

  rms:    0.177 Angstroms (50 lines)
  Derived pixel shift from input wcal for row: 899 01
  See identified lines.
  rms:    2.304
rejecting 58 points from 1828 total: 
  rms:    0.850
rejecting 59 points from 1828 total: 
  rms:    0.823
rejecting 59 points from 1828 total: 
  See 2D wavecal fit. Enter space in plot window to continue



OK, use the solution to make a wavelength map

In [87]:
t.clear()
t.tv(arc2d.wave)

Subtract out the central wavelength solution to see how the solution varies with row

In [88]:
dw=arc2d.wave-arc2d.wave[450]
t.tv(dw)  

In [89]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
obj=red.reduce(22)
t.tv(obj)
obj2d=trace.extract2d(obj)
wav.add_wave(obj2d)
t.tv(obj2d)
star=red.reduce(38)
star2d=trace.extract2d(star)
wav.add_wave(star2d)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_GCPilot02_red.0022.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_red.0038.fits
  subtracting overscan vector 
  subtracting overscan vector 


Here we rectify the image to have a constant wavelength scale (in log lambda). We choose the new scale based on the starting and ending wavelengths in the original image, and resample to get the same number of pixels.

If you preferred not to resample your object (or at least minimize the ressampling), you could resample the image to the wavelength array at the location of your object

In [92]:
wlim=(arc2d.wave[450,0],arc2d.wave[450,-1])
wnew=10.**np.linspace(np.log10(wlim[1]),np.log10(wlim[0]),4095)
print(wnew)
arc2d_rect=wav.correct(arc2d,wnew)
t.tv(arc2d)
t.tv(arc2d_rect)

[ 5497.80878063  5498.5599381   5499.31119819 ...,  9615.68205431
  9616.99583069  9618.30978658]


In [93]:
t.tv(obj2d)
obj2d_rect=wav.correct(obj2d,wnew)
star2d_rect=wav.correct(star2d,wnew)
t.tv(obj2d_rect)


OK, now we can extract in the wavelength rectified image for better sky subtraction. Create a new trace object with the rows= attribute to give the new extent of the image in rows, since our previous extraction limited the extraction to the length of the slit, as set by the rows attribute in the original trace object.

In [104]:
rows=[0,star2d_rect.shape[0]]
trace=spectra.Trace(sc0=int(star2d_rect.shape[1]/2.),rows=rows)
t.clear()
rows,fibers=trace.findpeak(star2d_rect)
print('rows: ',rows)
trace.trace(star2d_rect,rows,display=t)
ext=trace.extract(obj2d_rect,display=t,rad=10)
ext.add_wave(wnew)
ext_sub=trace.extract(obj2d_rect,rad=10,back=[[20,30]],display=t)
ext_sub.add_wave(wnew)

looking for peaks using 200 pixels around 2047, threshhold of 5.000000
rows:  [409]


/home/holtz/git/pyvista/python/pyvista/spectra.py:985: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:996: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<1))[0]



  See trace. Hit space bar to continue....
  See extraction window(s). Hit space bar to continue....

  See extraction window(s). Hit space bar to continue....
  See extraction window(s). Hit space bar to continue....


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(ext.wave,ext.data[0])
plt.plot(ext.wave,ext_sub.data[0])

CR rejection via multiple image stacking

<h4> KOSMOS blue channel

In [111]:
bwav=spectra.WaveCal('KOSMOS/KOSMOS_blue_waves.fits')
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arcs=red.sum([49,50,51])
arcec=trace.extract(arcs)
bwav.identify(arcec,plot=True)


  rms:    0.154 Angstroms (26 lines)
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_blue.0049.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_blue.0050.fits
  subtracting overscan vector 
  subtracting overscan vector 
  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_HeNeAr_blue.0051.fits
  subtracting overscan vector 
  subtracting overscan vector 
  combining data with sum....
  calculating uncertainty....



/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


  See identified lines.
  rms:    0.161 Angstroms (25 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.161 Anstroms
  input from plot window...



In [114]:
objec=trace.extract(red.reduce(26),display=None,rad=20)
bwav.add_wave(objec)
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

swav=copy.copy(bwav)
swav.model.fixed['c0']=False
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(bwav.model)
print(swav.model)

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_GCPilot02_blue.0026.fits
  subtracting overscan vector 
  subtracting overscan vector 

  See identified lines.
  rms:    0.127 Angstroms (3 lines)
  Input in plot window: 
       l : to remove all lines to left of cursor
       r : to remove all lines to right of cursor
       n : to remove line nearest cursor x position
       anything else : finish and return
  rms:    0.127 Anstroms
  input from plot window...

Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 3
Domain: (94.113632053360092, 3889.9479978108807)
Parameters:
          c0            c1            c2            c3     
    ------------- ------------- ------------- -------------
    5165.65355238 -1335.7037191 45.0756011611 13.5332345502
Model: Chebyshev1D
Inputs: ('x',)
Outputs: ('y',)
Model set size: 1
Degree: 3
Domain: (94.113632053360092, 3889.9479978108807)
Parameters:
          c0            c1            c2            c3     
 

In [119]:
flx=spectra.FluxCal(degree=5)
for im in [34,35,36] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    bwav.add_wave(starec)
    flx.addstar(starec[0],wave[0],file='flux/okestan/ffeige34.dat')

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_blue.0034.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  -0.233152718186


/home/holtz/git/pyvista/python/pyvista/spectra.py:986: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) )[0]
/home/holtz/git/pyvista/python/pyvista/spectra.py:997: RuntimeWarning: invalid value encountered in true_divide
  gd = np.where((~ymask) & (ysum/np.sqrt(yvar)>thresh) & (np.abs(res)<1))[0]




  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_blue.0035.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  0.171318784907
  Tracing row: 957

  Reading file: /home/users/adijeau/Documents/UT220107/KOSMOS_Feige34_blue.0036.fits
  subtracting overscan vector 
  subtracting overscan vector 
Using shift:  0.0600636186239
  Tracing row: 957



In [120]:
flx.response(plot=True)

In [126]:
plt.figure()
flx.correct(starec,starec.wave[0])
plt.plot(starec.wave[0],starec.data[0])